In [1]:
import tensorflow as tf
import numpy as np
from tdc.single_pred import Tox
from rdkit import Chem

2023-12-29 20:13:22.828692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 20:13:22.995982: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-29 20:13:23.896068: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.3/lib64
2023-12-29 20:13:23.896172: W te

In [2]:
def Chem_generator(smiles):
    res_list = []
    for i in tqdm(smiles):
        mol = Chem.MolFromSmiles(i)
        temp = []
        index = 0
        while(len(set(temp))!=4 and index != 100):
            index+=1
            temp.append(Chem.MolToSmiles(mol,doRandom=True))
        res_list+=list(set(temp))
    return res_list

In [11]:
from rdkit.Chem import MolFromSmiles, MolToSmiles
import atomInSmiles
from tqdm import tqdm
import atomInSmiles
def encode(smiles_list,labels):
    corpus = list(zip(smiles_list,labels))
    mols = []
    labels = []
    len_list = []
    res_labels = []
    for index,smiles in enumerate(corpus):
        try:
            mol = MolFromSmiles(smiles[0])
        except:
            continue
        if mol == None:
            continue
        mols.append(mol)
        labels.append(smiles[1])
        
    
    print(len(mols),len(labels))
    
    corpus = list(zip(mols,labels))
    print(mols[0])
    res = []
    for index1,mol in (enumerate(tqdm(corpus))):
        index = 0
        res_temp = []
        while(len(res_temp)<20 and index <100):
            try:
                random_smiles = MolToSmiles(mol[0], doRandom=True) # e.g 'C(C(=O)O)N' 
            except:
                print(mol[0])
                raise
            res_temp.append(random_smiles)
            res_temp = list(set(res_temp))
            index+=1
        res += res_temp
        res_labels += [mol[1]]*len(res_temp)
        len_list.append(len(res_temp))
        
    # mapping atomID into SMILES srting
    res_train = []


    for smiles in tqdm(res):
        tmp = MolFromSmiles(smiles)
        for atom in tmp.GetAtoms():
            atom.SetAtomMapNum(atom.GetIdx())
        smiles_1 = MolToSmiles(tmp) # 'C([C:1](=[O:2])[OH:3])[NH2:4]' 
        ais_tokens_1 = atomInSmiles.encode(smiles_1, with_atomMap=True).split(' ')
        res_train.append(ais_tokens_1)

    return res_train,res_labels,len_list

In [8]:
Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']

In [9]:
tox_train,tox_label = Tox(name='herg').get_data(format='DeepPurpose')

Found local copy...
Loading...
Done!


In [36]:
len(encode(tox_train[:5],tox_label[:5])[0])

100%|██████████| 100/100 [00:00<00:00, 458.74it/s]


100

In [16]:
def create_variable(var_name):
    globals()[var_name] = []
def get_variable_value(var_name):
    return globals().get(var_name)


In [38]:
import pickle
for tox in Tox_list:
    create_variable(tox)
    tox_train,tox_label = Tox(name=tox).get_data(format='DeepPurpose')
    temp_train = get_variable_value(tox)
    res = encode(tox_train,tox_label)
    temp_train.append(res)
    with open('./AIS_Tox_data/'+tox,'wb') as file:
        pickle.dump(temp_train,file)


Found local copy...
Loading...
Done!
100%|██████████| 268795/268795 [06:41<00:00, 669.20it/s]
Found local copy...
Loading...
Done!
100%|██████████| 141234/141234 [01:48<00:00, 1297.08it/s]
Found local copy...
Loading...
Done!
[13:46:48] WARNING: not removing hydrogen atom without neighbors
[13:46:48] WARNING: not removing hydrogen atom without neighbors
 86%|████████▋ | 11293/13092 [00:14<00:01, 1198.55it/s][13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:47:03] WARNING: not removing hydrogen atom without neighbors
[13:

In [27]:
from tdc.utils import retrieve_label_name_list
import pickle
Tox_list_label = ['herg_central','Tox21','ToxCast']
for tox in Tox_list_label:
    label_list = retrieve_label_name_list(tox)
    for label in label_list:

        create_variable(tox+'_'+label)
        tox_train,tox_label = Tox(name=tox,label_name=label).get_data(format='DeepPurpose')
        temp_train = get_variable_value(tox+'_'+label)
        temp_train.append(encode(tox_train,tox_label))
        with open('./AIS_Tox_data/'+tox+'_'+label,'wb') as file:
            pickle.dump(temp_train,file)

Found local copy...
Loading...
Done!
 13%|█▎        | 794780/6137266 [16:12<1:44:04, 855.48it/s] 

In [4]:
from tdc.single_pred import Tox
import pickle
from Module.tokenizer import Tokenizer

Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']
for name in Tox_list:
    data = Tox(name = name)
    split = data.get_split(method = 'scaffold')
    train,train_label = split['train'].values[:,1],split['train'].values[:,2]
    

Found local copy...
Loading...
Done!
100%|██████████| 13445/13445 [00:05<00:00, 2511.33it/s]
Found local copy...
Loading...
Done!
100%|██████████| 7385/7385 [00:01<00:00, 6285.09it/s]
Found local copy...
Loading...
Done!
100%|██████████| 655/655 [00:00<00:00, 3285.03it/s]
Found local copy...
Loading...
Done!
100%|██████████| 7278/7278 [00:01<00:00, 5238.30it/s]
Found local copy...
Loading...
Done!
100%|██████████| 475/475 [00:00<00:00, 3726.25it/s]
Found local copy...
Loading...
Done!
100%|██████████| 404/404 [00:00<00:00, 6761.71it/s]
Found local copy...
Loading...
Done!
100%|██████████| 280/280 [00:00<00:00, 3294.65it/s]
Found local copy...
Loading...
Done!
100%|██████████| 1478/1478 [00:00<00:00, 3161.20it/s]


In [ ]:
import pickle
Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']
for tox in Tox_list:
    data = Tox(name = tox)
    split = data.get_split(method = 'scaffold',frac=[0.8,0.2,0])
    train,train_label = split['train'].values[:,1],split['train'].values[:,2]
    val,val_label = split['valid'].values[:,1],split['valid'].values[:,2]
    test,test_label = split['test'].values[:,1],split['test'].values[:,2]
    print(val_label)
    train_res = encode(train,train_label)
    val_res = encode(val,val_label)
    test_res = encode(test,test_label)
    with open(f'./data_sets/{tox}_train','wb') as file:
        pickle.dump(train_res,file)
    with open(f'./data_sets/{tox}_val','wb') as file:
        pickle.dump(val_res,file)
    with open(f'./data_sets/{test}_val','wb') as file:
        pickle.dump(test_res,file)

In [4]:
import pickle
Tox_list = ['hERG_Karim','LD50_Zhu','hERG','AMES','DILI','Skin Reaction','Carcinogens_Lagunin','ClinTox']
for tox in Tox_list:
    data = Tox(name = tox)
    split = data.get_split(method = 'scaffold',frac=[0.8,0.2,0])
    train,train_label = split['train'].values[:,1],split['train'].values[:,2]
    val,val_label = split['valid'].values[:,1],split['valid'].values[:,2]
    test,test_label = split['test'].values[:,1],split['test'].values[:,2]
    print(val_label)
    train_res = encode(train,train_label)
    val_res = encode(val,val_label)
    test_res = encode(test,test_label)
    with open(f'./data_sets/{tox}_train','wb') as file:
        pickle.dump(train_res,file)
    with open(f'./data_sets/{tox}_val','wb') as file:
        pickle.dump(val_res,file)
    with open(f'./data_sets/{test}_val','wb') as file:
        pickle.dump(test_res,file)

Found local copy...
Loading...
Done!
100%|██████████| 13445/13445 [00:05<00:00, 2479.59it/s]


[1 0 1 ... 1 1 0]


 63%|██████▎   | 6790/10755 [00:09<00:05, 754.21it/s]

In [19]:
import pandas as pd
import glob
import pickle
# CSV 파일이 있는 폴더 경로 설정
folder_path = './Final_data/CLS/'

# 폴더 내의 모든 CSV 파일 경로 가져오기
csv_files = glob.glob(folder_path + '*.csv')
tox_names = []
for file in csv_files:
    name = file.split('/')[3]
    tox_names.append(name[:-4])

# 모든 CSV 파일을 담을 빈 데이터프레임 생성
all_data = pd.DataFrame()
train_Index = ['Smiles','smiles','SMILES']
label_Index = ['Property','expt','exp','p_np']
# 각 CSV 파일을 읽어와서 데이터프레임에 추가
for index,file_path in enumerate(csv_files):
    df = pd.read_csv(file_path)
    for t in train_Index:
        try:
            train_data=df[t]
        except:
            pass
    for l in label_Index:
        try:
            label_data = df[l]
        except:
            pass
    train_data = train_data.to_numpy()
    label_data = label_data.to_numpy()
# 모든 데이터가 병합된 데이터프레임 확인
    res = encode(train_data,label_data)
    with open('./Final_data_token/CLS/'+tox_names[index],'wb') as file:
        pickle.dump(res,file)

353 353


100%|██████████| 6696/6696 [00:07<00:00, 912.36it/s] 
[20:29:47] Explicit valence for atom # 1 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 6 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 6 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 11 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 12 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 5 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 5 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 5 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 5 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 5 N, 4, is greater than permitted
[20:29:47] Explicit valence for atom # 5 N, 4, is greater than permitted


2039 2039


100%|██████████| 40201/40201 [00:46<00:00, 868.12it/s] 


1427 1427


...

  3%|▎         | 1048/40202 [00:01<00:43, 899.44it/s][20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydrogen atom without neighbors
[20:48:39] WARNING: not removing hydr

8069 8069


100%|██████████| 161017/161017 [03:16<00:00, 821.49it/s] 


3219 3219


100%|██████████| 64378/64378 [01:40<00:00, 640.01it/s]


In [17]:
res[2]

[20,
 20,
 20,
 20,
 20,
 1,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 8,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 10,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 10,
 20,
 8,
 20,
 20,
 20,
 20,
 20,
 20,
 2,
 1,
 5,
 12,
 20,
 12,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 18,
 20,
 20,
 14,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 8,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 6,
 20,
 20,
 2,
 20,
 20,
 20,
 18,
 20,
 20,
 20,
 20,
 20,
 20,
 9,
 20,
 20,
 7,
 14,
 4,
 20,
 20,
 5,
 6,
 20,
 20,
 16,
 14,
 20,
 20,
 20,
 20,
 20,
 20,
 10,
 18,
 12,
 10,
 20,
 16,
 20,
 20,
 20,
 20,
 20,
 10,
 20,
 20,
 20,
 10,
 18,
 20,
 20,
 20,
 20,
 1,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,


In [6]:
df

,Name,Smiles,Property
0,BBB001,COc1ccc(cc1)C2Sc3ccccc3N(CCN(C)C)C(=O)C2OC(=O)C,1
1,BBB002,CC(C(=O)O)c1ccc(c(F)c1)c2ccccc2,1
2,BBB003,CC(C)(C)c1nc(cc(n1)C(F)(F)F)N2CCN(CCCSC3=NC=CC...,1
3,BBB004,CC1=C(CCN2CCC(=C(c3ccc(F)cc3)c4ccc(F)cc4)CC2)C...,1
4,BBB005,Clc1cccc(c1)N2CCN(CCCN3N=C4C=CC=CN4C3=O)CC2,1
...,...,...,...
348,BBB349,C=CC#N,0
349,BBB350,NC(=O)N1c2ccccc2C3OC3c4ccccc14,0
350,BBB351,CC(=O)Nc1ccc(O)cc1,0
351,BBB352,CC12CCC3C(CCc4cc(O)ccc34)C1CC(F)C2O,0


In [39]:
import pandas as pd
import glob
import pickle
# CSV 파일이 있는 폴더 경로 설정
folder_path = './Final_data/REG/'

# 폴더 내의 모든 CSV 파일 경로 가져오기
csv_files = glob.glob(folder_path + '*.csv')
tox_names = []
for file in csv_files:
    name = file.split('/')[3]
    tox_names.append(name[:-4])

# 모든 CSV 파일을 담을 빈 데이터프레임 생성
all_data = pd.DataFrame()
train_Index = ['Smiles','smiles','SMILES']
label_Index = ['Property','expt','exp','p_np']
# 각 CSV 파일을 읽어와서 데이터프레임에 추가
for index,file_path in enumerate(csv_files):
    df = pd.read_csv(file_path)
    for t in train_Index:
        try:
            train_data=df[t]
        except:
            pass
    for l in label_Index:
        try:
            label_data = df[l]
        except:
            pass
    train_data = train_data.to_numpy()
    label_data = label_data.to_numpy()
# 모든 데이터가 병합된 데이터프레임 확인
    res = encode(train_data,label_data)
    with open('./Final_data_token/CLS/'+tox_names[index],'wb') as file:
        pickle.dump(res,file)

642 642
642 642


100%|██████████| 9521/9521 [00:05<00:00, 1741.33it/s]


4200 4200
4200 4200


100%|██████████| 83973/83973 [01:49<00:00, 766.91it/s]


In [ ]:
import pickle
for tox in Tox_list:
    res = encode(tox_train,tox_label)
    temp_train.append(res)
    with open('./Final_data_token/'+tox+'.pkl','wb') as file:
        pickle.dump(temp_train,file)